In [1]:
#1. Librerías.
%run "../librerias.ipynb"

/home/gmasi/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/home/gmasi/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#2. Constantes.
%run "../constantes.ipynb"

In [3]:
#3. Funciones
%run "../funciones.ipynb"

In [4]:
#4. Configuración general.
#i. Configuración de jupysql.
%pip install duckdb-engine --quiet
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

#ii. Conexión a DuckDB.
%sql duckdb:///:memory:


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
#5. Lectura de datos.

In [6]:
%%sql
create or replace table competencia_02_crudo as
select
    *
from read_csv_auto("{{dataset_file_crudo}}")

,Success


In [7]:
#6. Calculo la clase ternaria.

In [8]:
%%sql
create or replace table competencia_02 as
with periodos as (
    select distinct foto_mes from competencia_02_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_02_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if(mes_2=1,'CONTINUA', -- Si mes_2 = 1, entonces CONTINUA, mas allá si se fue en el medio.
             if(mes_1=0,'BAJA+1', -- Si +1 no continua y +2 no es = 0 o NULL, entonces BAJA+1 (evaluar si +2 es Nulo?)
                if((mes_1=1) AND (mes_2=0),'BAJA+2', -- Continuo +1 pero no al +2
                   NULL) -- Acá incluyo si es NULL-NULL (ultimo mes en analisis),ó 1-NULL (anteultimo mes y continuaba).
             )
        ) as clase_ternaria
    from todo t
    left join competencia_02_crudo c using (numero_de_cliente, foto_mes)
) select
  * EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1

,Success


In [9]:
#7. Exportamos los resultados.

In [10]:
%sql COPY competencia_02 TO '{{dataset_file_clase_ternaria}}' (FORMAT CSV, HEADER)

,Success
